<a href="https://colab.research.google.com/github/KhurramDevOps/Quarter-02/blob/master/Rag_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project: LangChain RAG with Google Gemini Flash and Pinecone**

In [1]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.4 MB/s eta 0:00:00


In [2]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2


In [28]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/my-gemini-api-444713-262839e78ed5.json"

In [5]:
from IPython.display import Markdown, display
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('pinecone_api')
pc = Pinecone(api_key=pinecone_api_key)


## Resetting and Recreating the Index  
This snippet deletes the existing index named **"practise-rag-project"**, then recreates it with the same configuration. The new index is initialized for further operations.



In [6]:
index_name = "practice-rag-project"
pc.delete_index(index_name)

pc.create_index(
    name=index_name,
    dimension=768,
    metric='cosine',
    spec  = ServerlessSpec(cloud="aws",region="us-east-1"),
)

index = pc.Index(index_name)

## Generating Embeddings with Google Generative AI
In this snippet, we use the `GoogleGenerativeAIEmbeddings` module from LangChain to generate vector embeddings for the query **"Building a Rag project!"**. The embeddings are computed using the **"models/embedding-001"** model from Google's Generative AI API. The first five values of the embedding vector are displayed as a preview.


In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

GEMINI_KEY = userdata.get("GEMINI_API_KEY_2")

embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [8]:
vector = embeddings.embed_query("Building a Rag project! ")



In [9]:
vector[:5]

[0.005886509083211422,
 -0.01920737698674202,
 -0.01310189813375473,
 -0.03790365159511566,
 -0.003551947884261608]

## Creating a Vector Store with Pinecone and Documents  
This snippet initializes a Pinecone vector store with Google AI embeddings and creates ten documents with diverse content and metadata. These documents are ready for semantic search or AI-powered analysis in the vector store.


In [10]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [11]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I have chocolate chip pancake and scrambled eggs for breakfast",
    metadata={"source": "personal", "meal": "breakfast"}
)

document_2 = Document(
    page_content="LangChain is a framework for building applications with LLMs (Large Language Models), such as GPT. It provides tools to manage chains, agents, and memory for building more advanced AI applications.",
    metadata={"topic": "LangChain", "category": "AI Framework"}
)

document_3 = Document(
    page_content="Agentic AI refers to autonomous AI systems that are capable of decision-making, learning, and adaptation in real-world environments without needing constant human intervention.",
    metadata={"topic": "Agentic AI", "category": "Artificial Intelligence", "importance": "High"}
)

document_4 = Document(
    page_content="In the latest world news, a new tech company has developed an innovative AI that is able to solve real-world problems faster than previous models, pushing the boundaries of automation and efficiency.",
    metadata={"topic": "Tech News", "date": "2025-01-02", "company": "Innovative AI Company"}
)

document_5 = Document(
    page_content="The use of AI in healthcare is growing rapidly. Recent advancements in diagnostic AI tools are helping doctors identify diseases more accurately and faster, significantly improving patient outcomes.",
    metadata={"topic": "Healthcare AI", "category": "Medical Technology", "impact": "Positive"}
)

document_6 = Document(
    page_content="LangChain offers a wide range of tools to work with LLMs. This includes support for document search, question-answering systems, and memory management to make intelligent agents more effective in real-world tasks.",
    metadata={"topic": "LangChain", "category": "AI Tools", "use_case": "Advanced workflows"}
)

document_7 = Document(
    page_content="Agentic AI is becoming increasingly important in industries like autonomous vehicles, robotics, and smart cities, where real-time decision-making and adaptability are crucial for success.",
    metadata={"topic": "Agentic AI", "category": "Industry Applications", "industries": ["Autonomous Vehicles", "Robotics", "Smart Cities"]}
)

document_8 = Document(
    page_content="A new world record has been set for the fastest internet speed, with researchers breaking through previous limitations using advanced fiber-optic technology, promising faster and more efficient global communication.",
    metadata={"topic": "Tech News", "category": "Innovation", "record": "Fastest Internet Speed", "date": "2025-01-02"}
)

document_9 = Document(
    page_content="In a recent study, AI-powered chatbots have been shown to outperform human customer service agents in resolving technical issues, reducing wait times and improving customer satisfaction.",
    metadata={"topic": "AI in Customer Service", "category": "Business Technology", "impact": "High"}
)

document_10 = Document(
    page_content="LangChain continues to evolve with new integrations, including support for databases, APIs, and external data sources, enabling more complex and efficient workflows for AI applications.",
    metadata={"topic": "LangChain", "category": "Development", "features": ["Database Integration", "API Support", "External Data Sources"]}
)

documents = [
    document_1, document_2, document_3, document_4, document_5,
    document_6, document_7, document_8, document_9, document_10
]


In [12]:
len(  documents)

10

## Adding Documents to Vector Store and Performing Similarity Search  
This code assigns unique IDs to the documents, adds them to the Pinecone vector store, and performs a similarity search for the query **"What is agent?"**. The top result's content is displayed.


In [13]:
from uuid import uuid4
uuid4()

UUID('8f789aaa-d634-44fd-9421-546e82b46238')

In [14]:
uuids = [str(uuid4()) for i in range (len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['3767584e-bb08-4cc2-8d24-f8c8159a548b',
 'cf1736ac-3638-45df-89b4-7782bb27eeeb',
 '6184d565-7acb-4b7d-beeb-41b80bbf9071',
 '5179807e-f1e7-49e2-a44f-ee75586d3e04',
 '7d32e84a-2942-41e6-b223-04c29e4067b5',
 '45bfd385-6960-4409-a54f-65a647ddc283',
 'cc89eac1-dc7a-4ed2-9205-675d74acb020',
 'd96f68f5-acdf-4269-8b53-cf6f7c7e78cf',
 '8dd282c4-1e7a-4a9d-923e-789ed258d37a',
 'cd7bf8f1-3de7-45a3-a599-e87c3a75cede']

In [17]:
vector_result = vector_store.similarity_search(
    "What is agent?", k=7,)
print(vector_result[0].page_content)

Agentic AI refers to autonomous AI systems that are capable of decision-making, learning, and adaptation in real-world environments without needing constant human intervention.


## Generating Contextual Answers with Google Generative AI  
This snippet defines a function to retrieve relevant documents from the vector store and uses Google's **Gemini 2.0** model to generate an answer based on the provided context. The response is displayed in Markdown format.


In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate

In [19]:
def user_answer(question):

  vector_result = vector_store.similarity_search(question, k=5)

  llm = ChatGoogleGenerativeAI(
      api_key=GEMINI_KEY,
      model = "gemini-2.0-flash-exp",
      max_tokens= 100,
      temperature=0.7
  )

  prompt1= PromptTemplate(
      input_variables=["question"],
      template = "Using this data {vector_result} . Answer the following question: \n\n{question}"
  )
  chain1= prompt1 | llm

  final_answer = chain1.invoke({"vector_result": vector_result, "question": question})

  return final_answer

In [20]:
response = user_answer("What's the latest news?")




In [21]:
display(Markdown(response.content))

Based on the provided documents, here's a summary of the latest news:

*   **Innovative AI:** A new tech company has developed an innovative AI that can solve real-world problems faster than previous models. This is pushing the boundaries of automation and efficiency.
*   **Fastest Internet Speed:** A new world record has been set for the fastest internet speed, achieved through advanced fiber-optic technology. This promises faster and more efficient global communication.

Both of these news items are

In [22]:
response_1 = user_answer("What is Langchain and where it is used ?")


In [24]:
display(Markdown(response_1.content))

Based on the provided documents, here's what we know about LangChain and its uses:

**What is LangChain?**

*   LangChain is a framework designed for building applications that utilize Large Language Models (LLMs), such as GPT.
*   It provides tools to manage chains, agents, and memory, which are essential for creating more advanced AI applications.
*   It's a framework that's continuously evolving with new integrations.

**Where is LangChain

In [25]:
response_2 = user_answer("What i like in breakfast , Tell me in Key points")

In [26]:
display(Markdown(response_2.content))

Okay, based on the provided documents, here's the answer to your question in key points:

**Key Points about Breakfast:**

*   **Chocolate Chip Pancakes:** You had chocolate chip pancakes for breakfast.
*   **Scrambled Eggs:** You also had scrambled eggs for breakfast.
